In [7]:
import pandas as pd
import xlwings as xw
import json

In [ ]:
import pandas as pd
import xlwings as xw
import json

wb = xw.Book("/Users/erikingwersen/Desktop/fileshare/out/excel/out_interactive.xlsm")
sheet = wb.sheets['Inputs']

stockpiles = sheet['D2'].options(pd.DataFrame, expand='table').value.reset_index()
rename_dict = {"ID": "id", "Área": "yard", "Quantidade (ton)": "weightIni"
}
stockpiles = stockpiles.rename(columns=rename_dict).astype({col: int for col in rename_dict.values()})

yards = sheet['L2'].options(pd.DataFrame, expand='table').value.reset_index()

engine_cols = [col for col in yards.columns if any(ch.isnumeric() for ch in col)]
rename_dict = {col: "".join(ch for ch in col if ch.isnumeric()) for col in engine_cols}
engine_ids = list(rename_dict.values())

yards = yards.rename(columns=rename_dict).astype({"yard": int})
yards["rails"] = yards[engine_ids].apply(lambda row: [idx for idx, value in enumerate(row, 1) if pd.notna(value)], axis=1)
yards = yards.drop(columns=engine_ids)

stockpiles = stockpiles.merge(yards, on="yard", how="left")

engines = (
    sheet['P2'].options(pd.DataFrame, expand='table').value.reset_index()
    .rename(columns={"Veículo": "id", "Taxa (ton/min)": "speedReclaim"})
    .astype(int)
)


rename_dict = {"De": "from", "Para": "to"}
travel_speed = sheet['S2'].options(pd.DataFrame, expand='table').value.reset_index().rename(columns=rename_dict)

stockpiles_from_to_cols = list(rename_dict.values())
speed_cols = travel_speed.columns.difference(stockpiles_from_to_cols)
travel_speed["travel_time"] = travel_speed[speed_cols[0]]

for col in speed_cols[1:]:
    travel_speed["travel_time"] = travel_speed["travel_time"].fillna(travel_speed[col])

travel_times = travel_speed.drop(columns=speed_cols).pivot_table(index=["from"], columns=["to"], values=["travel_time"]).values.tolist()

output_info = sheet["A2"].options(pd.DataFrame, expand='table').value.reset_index()

outputs = [
    {
      "id": 1,
      "destination": 1,
      "weight": int(output_info.iloc[0, 1]),
      "quality": [
        {
          "parameter": "Fe",
          "minimum": 60,
          "maximum": 100,
          "goal": 65,
          "importance": 10
        },
        {
          "parameter": "SiO2",
          "minimum": 2.8,
          "maximum": 5.8,
          "goal": 5.8,
          "importance": 1000
        },
        {
          "parameter": "Al2O3",
          "minimum": 2.5,
          "maximum": 4.9,
          "goal": 4.9,
          "importance": 100
        },
        {
          "parameter": "P",
          "minimum": 0.05,
          "maximum": 0.07,
          "goal": 0.07,
          "importance": 100
        }
      ],
      "time": 600
    }
]
inputs = [
    {
      "id": 1,
      "weight": 0.0,
      "quality": [
        {
          "parameter": "Fe",
          "value": 60
        },
        {
          "parameter": "SiO2",
          "value": 1.5
        },
        {
          "parameter": "Al2O3",
          "value": 0.8
        },
        {
          "parameter": "P",
          "value": 1.0
        }
      ],
      "time": 8.0
    }
]

instance_data = {
    "info": ["Instance_Interactive", 1000, 1],
    "stockpiles": [],
    "engines": [],
    "inputs": inputs,
    "outputs": outputs,
    "distancesTravel": travel_times,
    "timeTravel": travel_times
}
for _, row in stockpiles.iterrows():
    sp = {
        "id": int(row["id"]),
        "position": int(row["id"]) - 1,
        "yard": int(row["yard"]),
        "rails": [int(r) for r in row["rails"]],
        "capacity": int(row["weightIni"]),
        "weightIni": int(row["weightIni"]),
        "qualityIni": [
            {"parameter": "Fe", "value": float(row["Fe"])},
            {"parameter": "SiO2", "value": float(row["SiO2"])},
            {"parameter": "Al2O3", "value": float(row["Al2O3"])},
            {"parameter": "P", "value": float(row["P"])}
        ]
    }
    instance_data["stockpiles"].append(sp)

all_yards = [int(y) for y in stockpiles["yard"].drop_duplicates().to_list()]
for _, row in engines.iterrows():
    eng = {
      "id": int(row["id"]),
      "speedStack": 0.0,
      "speedReclaim": int(row["speedReclaim"]),
      "posIni": int(row["id"]),
      "rail": int(row["id"]),
      "yards": all_yards
    }
    instance_data["engines"].append(eng)

# Write the data to the JSON file
with open('../tests/instance_11.json', 'w') as json_file:
    json.dump(instance_data, json_file, indent=2)

In [6]:
wb = xw.Book("/Users/erikingwersen/Desktop/fileshare/out/excel/out_interactive.xlsm")
sheet = wb.sheets['Inputs']

,id,yard,weightIni,Fe,SiO2,Al2O3,P
0,1,1,60000,10.3,2.3,4.5,0.05
1,2,1,45000,76.3,5.6,4.8,0.05
2,3,2,100000,87.3,3.7,5.0,0.05
3,4,2,50000,56.3,3.8,5.3,0.09
4,5,3,60000,70.5,4.0,5.1,0.07
5,6,3,75000,65.4,4.5,4.9,0.06
6,7,4,40000,75.0,4.3,5.0,0.08
7,8,4,45000,60.3,5.5,4.7,0.06
8,9,5,50000,68.3,3.2,4.8,0.07
9,10,5,55000,72.1,3.5,5.2,0.09


In [28]:
stockpiles = sheet['D2'].options(pd.DataFrame, expand='table').value.reset_index()
rename_dict = {
    "ID": "id",
    "Área": "yard",
    "Quantidade (ton)": "weightIni"
}
stockpiles = stockpiles.rename(columns=rename_dict).astype({col: int for col in rename_dict.values()})

yards = sheet['L2'].options(pd.DataFrame, expand='table').value.reset_index()

engine_cols = [col for col in yards.columns if any(ch.isnumeric() for ch in col)]
rename_dict = {
    col: "".join(ch for ch in col if ch.isnumeric())
    for col in engine_cols
}
engine_ids = list(rename_dict.values())
yards = yards.rename(columns=rename_dict).astype({"yard": int})
yards["rails"] = yards[engine_ids].apply(
    lambda row: [idx for idx, value in enumerate(row, 1) if pd.notna(value)], 
    axis=1
)
yards = yards.drop(columns=engine_ids)

stockpiles = stockpiles.merge(yards, on="yard", how="left")
stockpiles

,id,yard,weightIni,Fe,SiO2,Al2O3,P,rails
0,1,1,60000,10.3,2.3,4.5,0.05,"[1, 2]"
1,2,1,45000,76.3,5.6,4.8,0.05,"[1, 2]"
2,3,2,100000,87.3,3.7,5.0,0.05,[2]
3,4,2,50000,56.3,3.8,5.3,0.09,[2]
4,5,3,60000,70.5,4.0,5.1,0.07,"[1, 2]"
5,6,3,75000,65.4,4.5,4.9,0.06,"[1, 2]"
6,7,4,40000,75.0,4.3,5.0,0.08,[2]
7,8,4,45000,60.3,5.5,4.7,0.06,[2]
8,9,5,50000,68.3,3.2,4.8,0.07,"[1, 2]"
9,10,5,55000,72.1,3.5,5.2,0.09,"[1, 2]"


In [31]:
engines = sheet['P2'].options(pd.DataFrame, expand='table').value.reset_index()
engines = engines.rename(columns={"Veículo": "id", "Taxa (ton/min)": "speedReclaim"}).astype(int)
engines

,id,speedReclaim
0,1,3900
1,2,2600


In [50]:
rename_dict = {
    "De": "from",
    "Para": "to",
}
travel_speed = sheet['S2'].options(pd.DataFrame, expand='table').value.reset_index()
travel_speed = travel_speed.rename(columns=rename_dict)
stockpiles_from_to_cols = list(rename_dict.values())
speed_cols = travel_speed.columns.difference(stockpiles_from_to_cols)
travel_speed["travel_time"] = travel_speed[speed_cols[0]]
for col in speed_cols[1:]:
    travel_speed["travel_time"] = travel_speed["travel_time"].fillna(travel_speed[col])

travel_speed = travel_speed.drop(columns=speed_cols)
travel_times = travel_speed.pivot_table(index=["from"], columns=["to"], values=["travel_time"]).values.tolist()

In [48]:
output_info = sheet["A2"].options(pd.DataFrame, expand='table').value.reset_index()

outputs = [
    {
      "id": 1,
      "destination": 1,
      "weight": int(output_info.iloc[0, 1]),
      "quality": [
        {
          "parameter": "Fe",
          "minimum": 60,
          "maximum": 100,
          "goal": 65,
          "importance": 10
        },
        {
          "parameter": "SiO2",
          "minimum": 2.8,
          "maximum": 5.8,
          "goal": 5.8,
          "importance": 1000
        },
        {
          "parameter": "Al2O3",
          "minimum": 2.5,
          "maximum": 4.9,
          "goal": 4.9,
          "importance": 100
        },
        {
          "parameter": "P",
          "minimum": 0.05,
          "maximum": 0.07,
          "goal": 0.07,
          "importance": 100
        }
      ],
      "time": 600
    }
]
inputs = [
    {
      "id": 1,
      "weight": 0.0,
      "quality": [
        {
          "parameter": "Fe",
          "value": 60
        },
        {
          "parameter": "SiO2",
          "value": 1.5
        },
        {
          "parameter": "Al2O3",
          "value": 0.8
        },
        {
          "parameter": "P",
          "value": 1.0
        }
      ],
      "time": 8.0
    }
]

In [61]:
type(travel_times[0][0])

float

In [62]:
instance_data = {
    "info": ["Instance_Interactive", 1000, 1],
    "stockpiles": [],
    "engines": [],
    "inputs": inputs,
    "outputs": outputs,
    "distancesTravel": travel_times,
    "timeTravel": travel_times
}
for _, row in stockpiles.iterrows():
    sp = {
        "id": int(row["id"]),
        "position": int(row["id"]) - 1,
        "yard": int(row["yard"]),
        "rails": [int(r) for r in row["rails"]],
        "capacity": int(row["weightIni"]),
        "weightIni": int(row["weightIni"]),
        "qualityIni": [
            {"parameter": "Fe", "value": float(row["Fe"])},
            {"parameter": "SiO2", "value": float(row["SiO2"])},
            {"parameter": "Al2O3", "value": float(row["Al2O3"])},
            {"parameter": "P", "value": float(row["P"])}
        ]
    }
    instance_data["stockpiles"].append(sp)

all_yards = [int(y) for y in stockpiles["yard"].drop_duplicates().to_list()]
for _, row in engines.iterrows():
    eng = {
      "id": int(row["id"]),
      "speedStack": 0.0,
      "speedReclaim": int(row["speedReclaim"]),
      "posIni": int(row["id"]),
      "rail": int(row["id"]),
      "yards": all_yards
    }
    instance_data["engines"].append(eng)

In [63]:
# Write the data to the JSON file
with open('../tests/instance_11.json', 'w') as json_file:
    json.dump(instance_data, json_file, indent=2)

In [2]:
def export_to_json():
    # Open the workbook and select the Inputs sheet
    wb = xw.Book.caller()  # Connect to the workbook that called this function
    sheet = wb.sheets['Inputs']

    # Initialize instance data
    instance_data = {
        "info": ["Instance_Interactive", 1000, 1],
        "stockpiles": [],
        "engines": [],
        "inputs": [],
        "outputs": [],
        "distancesTravel": travel_times,
        "timeTravel": travel_times
    }

    # Read Stockpiles data
    row = 3  # Starting row for stockpiles, adjust if necessary
    while sheet.range(f"D{row}").value:
        stockpile = {
            "id": int(sheet.range(f"D{row}").value),
            "position": int(sheet.range(f"D{row}").value - 1),
            "yard": int(sheet.range(f"E{row}").value),
            "rails": [1, 2],
            "capacity": int(sheet.range(f"F{row}").value),
            "weightIni": int(sheet.range(f"F{row}").value),
            "qualityIni": [
                {"parameter": "Fe", "value": float(sheet.range(f"G{row}").value)},
                {"parameter": "SiO2", "value": float(sheet.range(f"H{row}").value)},
                {"parameter": "Al2O3", "value": float(sheet.range(f"I{row}").value)},
                {"parameter": "P", "value": float(sheet.range(f"J{row}").value)}
            ]
        }
        instance_data["stockpiles"].append(stockpile)
        row += 1

    # Read Engines data
    row = 3  # Adjust if necessary, find the first row of engines data
    engine_ids = sheet.range(f"P{row}:P{row+2}").value  # Assuming up to 20 rows
    speed_reclaims = sheet.range(f"Q{row}:Q{row+2}").value

    yards = {engine_id: [] for engine_id in engine_ids}

    while sheet.range(f"L{row}").value:
        # Find the yards accessible by the engine
        if sheet.range(f"M{row}").value == "x":
            yards[engine_ids[0]].append(sheet.range(f"L{row}").value)
        if sheet.range(f"N{row}").value == "x":
            yards[engine_ids[1]].append(sheet.range(f"L{row}").value)

    for engine_id, speed_reclaim in zip(engine_ids, speed_reclaims):
        engine_yards = yards[engine_id]
        for stockpile in instance_data["stockpiles"]:
            if stockpile["yard"] in engine_yards:
                posIni = stockpile["id"]
                break

        engine = {
            "id": int(engine_id),
            "speedStack": 0.0,
            "speedReclaim": speed_reclaim,
            "posIni": posIni,  # Set to first stockpile ID if any yards
            "rail": [1, 2],  # Assuming rail is in column O
            "yards": engine_yards
        }
        instance_data["engines"].append(engine)

    # Read the Inputs section similarly if needed
    
    # Read Time Travel and Distances Travel data
    # Assuming these columns have a consistent number of rows as stockpiles
    row = 3  # Start row for stockpiles
    times = []
    while sheet.range(f"S{row}").value:
        time_row = [sheet.range(f"S{row}").value, sheet.range(f"T{row}").value]
        time_row.extend([sheet.range(f"U{row}").value, sheet.range(f"V{row}").value])
        times.append([t for t in time_row if t is not None])  # Removing None values
        row += 1

    # Fill timeTravel and distancesTravel with the same matrix
    instance_data["timeTravel"] = times
    instance_data["distancesTravel"] = times

    # Write the data to the JSON file
    with open('instance_9.json', 'w') as json_file:
        json.dump(instance_data, json_file, indent=2)

    xw.alert('JSON file created successfully!')


In [5]:
xw.Book("/Users/erikingwersen/Desktop/fileshare/out/excel/out_interactive.xlsm").set_mock_caller()
export_to_json()

KeyboardInterrupt: 

In [42]:
from __future__ import annotations
from typing import List, Dict, Any
import subprocess
import logging
from pathlib import Path
import json

import numpy as np
import pandas as pd

import xlwings as xw


def check_is_file(*filepaths):
    files_not_found = []
    for filepath in filepaths:
        _filepath = Path(filepath)
        if not _filepath.is_file():
            files_not_found.append(filepath)
    if len(files_not_found) > 0:
        plural = "" if len(files_not_found) == 1 else "s"
        past_tense_form = "was" if len(files_not_found) == 1 else "were"
        exception_message = (
            f"The following file{plural} {past_tense_form}n't found: "
            + ", ".join(files_not_found)
        )
        raise FileNotFoundError(exception_message)


# Helper function to autofit column widths with a minimum width
def autofit_columns(ws):
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        adjusted_width = max(max_length + 2, 10)  # Minimum width set to 10
        ws.column_dimensions[column].width = adjusted_width


# Convert input and output JSON files to dataframes
def explode_quality_rows(
    df: pd.DataFrame,
    quality_col_prefix: str = "quality_",
) -> pd.DataFrame:
    """
    Explode the `'quality_*'` columns into rows.

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe with the output specifications with the quality columns
        that contain dictionaries of quality parameters to be extracted.
    quality_col_prefix : str, default='quality_'
        The prefix of the quality columns that contain the output pile specifications
        that need to be extracted into different columns.

    Returns
    -------
    pd.DataFrame
        The `pandas.DataFrame` with the quality dictionaries extracted into
        new columns.
    """
    # Create a new DataFrame to store exploded rows
    exploded_df = pd.DataFrame()

    for index, row in df.iterrows():
        quality_dict_list = [
            value for key, value in row.items() if key.startswith(quality_col_prefix)
        ]

        # Convert the list of quality dictionaries into a DataFrame
        quality_df = pd.DataFrame(quality_dict_list)

        # Repeat the original columns for each exploded row
        repeated_columns = pd.DataFrame(
            [
                row.drop(
                    labels=[
                        col for col in df.columns if col.startswith(quality_col_prefix)
                    ]
                )
            ]
            * len(quality_df)
        )

        # Concatenate the repeated columns with the quality columns
        exploded_row_df = pd.concat(
            [
                repeated_columns.reset_index(drop=True),
                quality_df.reset_index(drop=True),
            ],
            axis=1,
        )

        # Append to the exploded DataFrame
        exploded_df = pd.concat([exploded_df, exploded_row_df], ignore_index=True)

    return exploded_df


def assign_engines_to_stockpiles(
    stockpiles_df: pd.DataFrame, engines_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Assign engines to stockpiles based on matching yards and rails.

    Parameters
    ----------
    stockpiles_df : pd.DataFrame
        A `pandas.DataFrame` containing stockpile information including
        'rails' and 'yard'.
    engines_df : pd.DataFrame
        A `pandas.DataFrame` containing engine information including 'yards' and 'rail'.

    Returns
    -------
    pd.DataFrame
        Updated `stockpiles_df` with an 'engines' column listing the assigned engine IDs.
    """
    stockpiles_df["engines"] = [[] for _ in range(stockpiles_df.shape[0])]

    for idx, stockpile in stockpiles_df.iterrows():
        assigned_engines = [
            eng_row["id"]
            for _, eng_row in engines_df.iterrows()
            if stockpile["yard"] in eng_row["yards"]
            and eng_row["rail"] in stockpile["rails"]
        ]
        stockpiles_df.at[idx, "engines"] = assigned_engines

    return stockpiles_df


def extract_quality_ini_values(
    stockpiles_df: pd.DataFrame, quality_prefix: str = "qualityIni"
) -> pd.DataFrame:
    """
    Extract initial quality values from nested dictionaries in the stockpiles DataFrame
    and add them as new columns.

    Parameters
    ----------
    stockpiles_df : pd.DataFrame
        A `pandas.DataFrame` containing stockpile information, including quality parameters.
    quality_prefix : str, default='qualityIni'
        Prefix used in column names for quality-related information.

    Returns
    -------
    pd.DataFrame
        Updated `stockpiles_df` with quality parameters extracted as individual columns.
    """
    quality_cols = stockpiles_df.columns[
        stockpiles_df.columns.str.startswith(quality_prefix)
    ]
    quality_ini_dict = {}

    for column in quality_cols:
        for idx, row in stockpiles_df.iterrows():
            parameter = row[column]["parameter"]
            value = row[column]["value"]
            quality_ini_dict.setdefault(parameter, []).append(value)

    # Add the extracted quality values as new columns and drop the original quality columns
    stockpiles_df = pd.concat(
        [stockpiles_df, pd.DataFrame(quality_ini_dict, index=stockpiles_df.index)],
        axis=1,
    ).drop(columns=quality_cols, errors="ignore")

    return stockpiles_df


def process_stockpiles_and_engines(
    stockpiles_df: pd.DataFrame, engines_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Process stockpiles and engines by assigning engines to stockpiles and extracting initial quality values.

    Parameters
    ----------
    stockpiles_df : pd.DataFrame
        A `pandas.DataFrame` containing stockpile information.
    engines_df : pd.DataFrame
        A `pandas.DataFrame` containing engine information.

    Returns
    -------
    pd.DataFrame
        Processed `stockpiles_df` with engines assigned and quality values extracted.
    """
    stockpiles_df = assign_engines_to_stockpiles(stockpiles_df, engines_df)
    stockpiles_df = extract_quality_ini_values(stockpiles_df)
    return stockpiles_df


def travel_time(grp: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the travel time between consecutive events within a group.

    This function calculates the time between the end of one event and the start of the next event
    within a grouped DataFrame. If the group contains only one event, the travel time is set to 0.

    Parameters
    ----------
    grp : pd.DataFrame
        A `pandas.DataFrame` containing at least 'start_time' and 'end_time'
        columns. The DataFrame is expected to be pre-grouped by a relevant key
        before being passed to this function.

    Returns
    -------
    pd.DataFrame
        A `pandas.DataFrame` with a single column 'travel_time',
        containing the calculated travel times between consecutive events.
        The index of the returned DataFrame matches the input DataFrame.
    """
    if len(grp) == 1:
        return pd.DataFrame(
            {"travel_time": [grp["start_time"].values[0]]}, index=grp.index
        )
    grp = grp.sort_values(["end_time"])
    end_time = None
    res = []
    for _, row in grp.iterrows():
        _end_time = row["end_time"]
        if end_time is None:
            res.append(row["start_time"])
        else:
            res.append(row["start_time"] - end_time)
        end_time = _end_time
    return pd.DataFrame({"travel_time": res}, index=grp.index)


def json_input_output_to_excel(
    json_input_path: str | Path,
    json_output_path: str | Path,
    excel_path: str | Path | None = None,
):
    check_is_file(json_input_path, json_output_path)
    
    if excel_path is None:
        excel_filename = Path(json_output_path).with_suffix(".xlsx").name
        excel_folder_path = Path(json_output_path).parent.parent.joinpath("excel")
        excel_path = str(excel_folder_path.joinpath(excel_filename))
    else:
        excel_folder_path = Path(excel_path).parent

    excel_folder_path.mkdir(exist_ok=True, parents=True)

    # Load JSON files
    # Input file
    with open(json_input_path) as fh:
        instance_data = json.load(fh)
    
    # Output file
    with open(json_output_path) as fh:
        output_data = json.load(fh)
    
    info_df = pd.DataFrame([instance_data["info"]], columns=["Instance_Name", "Capacity", "Yard"])
    engines_df = pd.DataFrame(instance_data["engines"])
    stockpiles_df = pd.DataFrame(instance_data["stockpiles"])
    stockpiles_quality_df = pd.json_normalize(stockpiles_df.pop("qualityIni"), sep="_").add_prefix("qualityIni_")
    stockpiles_df = pd.concat([stockpiles_df, stockpiles_quality_df], axis=1)
    stockpiles_df = process_stockpiles_and_engines(stockpiles_df, engines_df)
    
    inputs_df = pd.DataFrame(instance_data["inputs"])
    inputs_quality_df = pd.json_normalize(inputs_df.pop("quality"), sep="_").add_prefix("quality_")
    inputs_df = pd.concat([inputs_df, inputs_quality_df], axis=1)
    
    # Convert instance_1.json to DataFrames
    outputs_df = pd.DataFrame(instance_data["outputs"])
    outputs_quality_df = pd.json_normalize(outputs_df.pop("quality"), sep="_").add_prefix("quality_")
    outputs_df = pd.concat([outputs_df, outputs_quality_df], axis=1)
    
    # Explode the outputs_df
    outputs_df = explode_quality_rows(outputs_df, quality_col_prefix="quality_").drop(
        columns=["time"], errors="ignore"
    )
    distances_travel_df = pd.DataFrame(instance_data["distancesTravel"])
    time_travel_df = pd.DataFrame(instance_data["timeTravel"])
    
    time_travel_df.columns += 1
    time_travel_df.index += 1
    
    distances_travel_df.columns += 1
    distances_travel_df.index += 1
    
    from_to_list = []
    for col in time_travel_df.columns:
        for idx in time_travel_df.index:
            from_to_list.append([f"{col} -> {idx}", time_travel_df.loc[idx, col]])
    from_to_df = pd.DataFrame(from_to_list, columns=["from_to", "duration"])
    
    engines_df[from_to_df["from_to"].to_list()] = -1
    engines_df[from_to_df["from_to"].to_list()] = engines_df[from_to_df["from_to"].to_list()].astype(float)
    
    for idx, row in engines_df.iterrows():
        yards = row["yards"]
        rail = row["rail"]
        stockpiles = []
        for _, stockpile_row in stockpiles_df.iterrows():
            if stockpile_row["yard"] in yards and rail in stockpile_row["rails"]:
                stockpiles.append(stockpile_row["id"])
        for start_stockpile in stockpiles:
            for end_stockpile in stockpiles:
                column_name = f"{start_stockpile} -> {end_stockpile}"
                duration = from_to_df.loc[
                    from_to_df["from_to"] == column_name, "duration"
                ].values[0]
                engines_df.loc[engines_df.index == idx, column_name] = duration
    
    engines_df[from_to_df["from_to"].to_list()] = engines_df[from_to_df["from_to"].to_list()].replace(-1, "")
    
    objective_df = pd.DataFrame([{"Objective": output_data["objective"], "Gap": output_data["gap"][0]}])
    stacks_df = pd.DataFrame(output_data["stacks"])
    reclaims_df = pd.DataFrame(output_data["reclaims"])
    
    outputs_df_out = pd.DataFrame(output_data["outputs"])
    outputs_quality_df_out = pd.json_normalize(outputs_df_out.pop("quality"), sep="_").add_prefix("quality_")
    
    outputs_df_out = pd.concat([outputs_df_out, outputs_quality_df_out], axis=1)
    outputs_df_out = explode_quality_rows(outputs_df_out, quality_col_prefix="quality_")
    
    if not stacks_df.empty:
        stacks_df["end_time"] = stacks_df["start_time"] + stacks_df["duration"]
        stacks_df["operation"] = "stack"
    
    if not reclaims_df.empty:
        reclaims_df["end_time"] = reclaims_df["start_time"] + reclaims_df["duration"]
        reclaims_df["operation"] = "reclaim"
    
    operations_df = (
        pd.concat([xdf for xdf in [stacks_df, reclaims_df] if not xdf.empty])
        .astype({"weight": int})
        .sort_values(["engine", "start_time"])
        .assign(
            travel_time=lambda xdf: (
                xdf.groupby("engine", as_index=False).apply(travel_time)
            )["travel_time"].reset_index(level=0, drop=True)
        )
    )
    
    stockpiles_final_df = (
        stockpiles_df.merge(
            operations_df.rename(columns={"weight": "weightFinal"})
            .groupby("stockpile")["weightFinal"]
            .sum(),
            left_on="id",
            right_index=True,
            how="left",
        )
        .fillna({"weightFinal": 0})
        .astype({"weightFinal": int})
        .assign(weightFinal=lambda xdf: xdf["weightIni"] - xdf["weightFinal"])
    )
    
    quality_cols = stockpiles_df.columns.intersection(["Fe", "SiO2", "Al2O3", "P", "+31.5", "-6.3"]).to_list()
    
    operations_df = operations_df.merge(
        stockpiles_df.rename(columns={"id": "stockpile"})[["stockpile", "weightIni", *quality_cols]],
        on="stockpile",
        how="left",
    ).assign(weightFinal=lambda xdf: xdf["weightIni"] - xdf["weight"])
    
    final_output_row = [
        operations_df["weight"].sum(),
        "output",
        operations_df["engine"].unique().tolist(),
        operations_df["start_time"].min(),
        operations_df["end_time"].max(),
        1,
        operations_df["end_time"].max(),
        "output_stack",
        operations_df["travel_time"].sum(),
        operations_df["weightFinal"].sum(),
    ]
    
    for quality_col in quality_cols:
        final_output_row.append(
            (
                (
                    operations_df["weight"]
                    * operations_df[quality_col]
                    / operations_df["weight"].sum()
                ).sum()
            )
        )
    operations_df = pd.concat(
        [
            operations_df,
            pd.DataFrame(
                {
                    col: [value]
                    for col, value in zip(operations_df.columns, final_output_row)
                }
            ),
        ],
        axis=0,
    )
    
    required_weight = operations_df.loc[operations_df["stockpile"] == "output", "weight"].values[0]
    infos_gerais = pd.DataFrame(
        {
            "Variável": ["Peso Carregamento"],
            "Valor": [required_weight]
        }
    )
    
    engines_yards = (
        stockpiles_final_df[["yard", "engines"]]
        .astype({"engines": str})
        .drop_duplicates()
        .assign(
            engines=lambda xdf: xdf["engines"]
            .str.replace("[", "")
            .str.replace("]", "")
            .str.replace(" ", "")
            .str.replace(",", "")
            .apply(list)
        )
    )
    all_engines = list(
        sorted(set([engine for engines in engines_yards["engines"] for engine in engines]))
    )
    for engine in all_engines:
        engines_yards[f"Veículo {engine}"] = engines_yards["engines"].apply(lambda value: "x" if engine in value else "")
    
    engines_yards = engines_yards.drop(columns=["engines"]).rename({"yard": "Área"})
    
    rename_dict = {
        "id": "ID",
        "yard": "Área",
        "weightIni": "Quantidade (ton)",
    }
    final_cols = [*list(rename_dict.values()), *quality_cols]
    stockpiles_final_df = stockpiles_final_df.rename(columns=rename_dict)[final_cols]
    
    load_rates = (
        engines_df[["id", "speedReclaim"]]
        .astype({"speedReclaim": int})
        .rename(columns={"id": "Veículo", "speedReclaim": "Taxa (ton/min)"})
    )
    
    from_to_cols = [col for col in engines_df.columns if "->" in col]
    travel_times_dict = {
        "De": [],
        "Para": [],
    }
    for from_to in from_to_cols:
        from_stockpile, to_stockpile = from_to.split(" -> ")
        travel_times_dict["De"].append(from_stockpile)
        travel_times_dict["Para"].append(to_stockpile)
        for engine in all_engines:
            vehicle_travel_times = travel_times_dict.get(f"Veículo {engine}", [])
            engine_row = engines_df.loc[engines_df["id"] == int(engine)]
            time_travel = engine_row[from_to].values[0]
            vehicle_travel_times.append(time_travel)
            travel_times_dict[f"Veículo {engine}"] = vehicle_travel_times
    
    travel_times_df = pd.DataFrame(travel_times_dict).astype({"De": int, "Para": int})
    
    rename_dict = {
        "engine": "Veículo",
        "stockpile": "Pilha",
        "weightIni": "Peso Inicial Pilha",
        "weightFinal": "Peso Final Pilha",
        "weight": "Carregamento (ton)",
        "start_time": "Início",
        "end_time": "Fim",
        "duration": "Tempo Carregamento",
        "travel_time": "Tempo Deslocamento",
    }
    operations_df = operations_df.rename(columns=rename_dict)[[*list(rename_dict.values()), *quality_cols]]
    operations_df[quality_cols] = operations_df[quality_cols].round(2)
    operations_df = operations_df.fillna("")
    
    outputs_df_out["check"] = np.where(
        (outputs_df_out["value"] >= outputs_df_out["minimum"]) & 
        (outputs_df_out["value"] <= outputs_df_out["maximum"]),
        True,
        False,
    )
    rename_dict = {
        "parameter": "Elemento",
        "value": "Valor",
        "minimum": "Mínimo",
        "maximum": "Máximo",
        "goal": "Meta",
        "check": "Check",
    }
    outputs_df_out = outputs_df_out.rename(columns=rename_dict)[list(rename_dict.values())]
    return operations_df, outputs_df_out


def run_pyblend_command(input_json: str, output_json: str, algorithm: str = "lahc") -> None:
    """Runs the pyblend command with the specified input and output JSON files.

    Parameters
    ----------
    input_json : str
        The input JSON file path.
    output_json : str
        The output JSON file path.
    algorithm : str, optional
        The algorithm to be used, by default "lahc".

    Raises
    ------
    RuntimeError
        If the command fails to execute successfully.
    """
    command = ["python", "../pyblend", input_json, output_json, "-algorithm", algorithm]
    
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        logging.info("Command executed successfully. Output:\n%s", result.stdout)
    except subprocess.CalledProcessError as e:
        logging.error("Command failed with error:\n%s", e.stderr)
        raise RuntimeError(f"Command failed: {e.stderr}")


class ExcelDataExtractor:
    """Class for extracting data from Excel sheets using xlwings.

    Attributes
    ----------
    workbook_path : str
        The path to the Excel workbook.
    sheet_name : str
        The name of the sheet from which data is extracted.

    Methods
    -------
    extract_dataframe(range: str, expand: bool = True) -> pd.DataFrame:
        Extracts a DataFrame from a specified range in the sheet.
    """

    def __init__(self, workbook_path: str, sheet_name: str):
        """Initialize the ExcelDataExtractor with workbook path and sheet name.

        Parameters
        ----------
        workbook_path : str
            The path to the Excel workbook.
        sheet_name : str
            The name of the sheet from which data is extracted.
        """
        self.wb = xw.Book(workbook_path)
        self.sheet = self.wb.sheets[sheet_name]

    def extract_dataframe(self, range: str, expand: bool = True) -> pd.DataFrame:
        """Extracts a DataFrame from a specified range in the sheet.

        Parameters
        ----------
        range : str
            The cell range to start extracting data from.
        expand : bool, optional
            Whether to expand the range to a table, by default True.

        Returns
        -------
        pd.DataFrame
            Extracted data as a pandas DataFrame.
        """
        return self.sheet[range].options(pd.DataFrame, expand='table' if expand else None).value.reset_index()


class StockpileProcessor:
    """Class for processing stockpile data.

    Methods
    -------
    process(stockpiles: pd.DataFrame, yards: pd.DataFrame, rename_dict: Dict[str, str]) -> pd.DataFrame:
        Processes and merges stockpile and yard data.
    """

    @staticmethod
    def process(stockpiles: pd.DataFrame, yards: pd.DataFrame, rename_dict: Dict[str, str]) -> pd.DataFrame:
        """Processes and merges stockpile and yard data.

        Parameters
        ----------
        stockpiles : pd.DataFrame
            DataFrame containing stockpile information.
        yards : pd.DataFrame
            DataFrame containing yard information.
        rename_dict : Dict[str, str]
            Dictionary for renaming columns.

        Returns
        -------
        pd.DataFrame
            Merged DataFrame of stockpiles and yards.
        """
        stockpiles = stockpiles.rename(columns=rename_dict).astype({col: int for col in rename_dict.values()})
        yards = StockpileProcessor._process_yards(yards)
        return stockpiles.merge(yards, on="yard", how="left")

    @staticmethod
    def _process_yards(yards: pd.DataFrame) -> pd.DataFrame:
        """Processes the yard data to extract rails information.

        Parameters
        ----------
        yards : pd.DataFrame
            DataFrame containing yard information.

        Returns
        -------
        pd.DataFrame
            Processed DataFrame with rails information.
        """
        engine_cols = [col for col in yards.columns if any(ch.isnumeric() for ch in col)]
        rename_dict = {col: "".join(ch for ch in col if ch.isnumeric()) for col in engine_cols}
        engine_ids = list(rename_dict.values())

        yards = yards.rename(columns=rename_dict).astype({"yard": int})
        yards["rails"] = yards[engine_ids].apply(
            lambda row: [idx for idx, value in enumerate(row, 1) if pd.notna(value)], axis=1)
        return yards.drop(columns=engine_ids)


class TravelSpeedProcessor:
    """Class for processing travel speed data.

    Methods
    -------
    process(travel_speed: pd.DataFrame, rename_dict: Dict[str, str]) -> List[List[float]]:
        Processes travel speed data into a list of travel times.
    """

    @staticmethod
    def process(travel_speed: pd.DataFrame, rename_dict: Dict[str, str]) -> List[List[float]]:
        """Processes travel speed data into a list of travel times.

        Parameters
        ----------
        travel_speed : pd.DataFrame
            DataFrame containing travel speed data.
        rename_dict : Dict[str, str]
            Dictionary for renaming columns.

        Returns
        -------
        List[List[float]]
            Nested list representing travel times between locations.
        """
        travel_speed = travel_speed.rename(columns=rename_dict)
        stockpiles_from_to_cols = list(rename_dict.values())
        speed_cols = travel_speed.columns.difference(stockpiles_from_to_cols)

        travel_speed["travel_time"] = travel_speed[speed_cols[0]]
        for col in speed_cols[1:]:
            travel_speed["travel_time"] = travel_speed["travel_time"].fillna(travel_speed[col])

        return travel_speed.drop(columns=speed_cols).pivot_table(
            index=["from"], columns=["to"], values=["travel_time"]).values.tolist()


class InstanceDataBuilder:
    """Class for building the instance data JSON structure.

    Methods
    -------
    build_instance_data
        Builds the instance data structure.
    """

    @staticmethod
    def build_instance_data(
        stockpiles: pd.DataFrame,
        engines: pd.DataFrame,
        travel_times: List[List[float]],
        inputs: List[Dict[str, Any]],
        outputs: List[Dict[str, Any]],
    ) -> Dict[str, Any]:
        """Builds the instance data structure.

        Parameters
        ----------
        stockpiles : pd.DataFrame
            DataFrame containing stockpile information.
        engines : pd.DataFrame
            DataFrame containing engine information.
        travel_times : List[List[float]]
            Nested list representing travel times between locations.
        inputs : List[Dict[str, Any]]
            List of dictionaries representing input data.
        outputs : List[Dict[str, Any]]
            List of dictionaries representing output data.

        Returns
        -------
        Dict[str, Any]
            The constructed instance data structure.
        """
        instance_data = {
            "info": ["Instance_Interactive", 1000, 1],
            "stockpiles": [],
            "engines": [],
            "inputs": inputs,
            "outputs": outputs,
            "distancesTravel": travel_times,
            "timeTravel": travel_times
        }

        for _, row in stockpiles.iterrows():
            sp = {
                "id": int(row["id"]),
                "position": int(row["id"]) - 1,
                "yard": int(row["yard"]),
                "rails": [int(r) for r in row["rails"]],
                "capacity": int(row["weightIni"]),
                "weightIni": int(row["weightIni"]),
                "qualityIni": [
                    {"parameter": "Fe", "value": float(row["Fe"])},
                    {"parameter": "SiO2", "value": float(row["SiO2"])},
                    {"parameter": "Al2O3", "value": float(row["Al2O3"])},
                    {"parameter": "P", "value": float(row["P"])}
                ]
            }
            instance_data["stockpiles"].append(sp)

        all_yards = [int(y) for y in stockpiles["yard"].drop_duplicates().to_list()]
        for _, row in engines.iterrows():
            eng = {
                "id": int(row["id"]),
                "speedStack": 0.0,
                "speedReclaim": int(row["speedReclaim"]),
                "posIni": int(row["id"]),
                "rail": int(row["id"]),
                "yards": all_yards
            }
            instance_data["engines"].append(eng)

        return instance_data


def update_excel_sheets(operations_df: pd.DataFrame, outputs_df_out: pd.DataFrame, excel_file: str) -> None:
    """Update the Excel sheets with the new data.

    Parameters
    ----------
    operations_df : pd.DataFrame
        DataFrame containing operation results.
    outputs_df_out : pd.DataFrame
        DataFrame containing output check results.
    excel_file : str
        Path to the Excel file to be updated.
    """
    wb = xw.Book(excel_file)
    resultados_sheet = wb.sheets['Resultados']
    check_res_sheet = wb.sheets['Check Restrições']

    # Clear existing data
    resultados_sheet.clear_contents()
    check_res_sheet.clear_contents()

    # Write new data
    operations_df = operations_df.fillna('')
    operations_df.iloc[-1, 0] = operations_df.iloc[0, -1].astype(str)
    operations_df = operations_df.set_index(operations_df.columns[0])
    
    resultados_sheet.range("A1").value = operations_df
    check_res_sheet.range("A1").value = outputs_df_out.fillna('').set_index(outputs_df_out.columns[0])


def main(
    excel_filepath: str,
    instance_json_path: str,
    output_json_path: str,
):
    # Initialize the ExcelDataExtractor with workbook path and sheet name
    extractor = ExcelDataExtractor(excel_filepath, "Inputs")

    # Extract data from the Excel sheet
    stockpiles_df = extractor.extract_dataframe('D2')
    yards_df = extractor.extract_dataframe('L2')
    engines_df = extractor.extract_dataframe('P2').rename(columns={"Veículo": "id", "Taxa (ton/min)": "speedReclaim"}).astype(int)
    travel_speed_df = extractor.extract_dataframe('S2')

    # Process stockpile and yard data
    stockpiles = StockpileProcessor.process(stockpiles_df, yards_df, {"ID": "id", "Área": "yard", "Quantidade (ton)": "weightIni"})

    # Process travel speed data
    travel_times = TravelSpeedProcessor.process(travel_speed_df, {"De": "from", "Para": "to"})

    output_info = extractor.extract_dataframe("A2")
    outputs = [
        {
            "id": 1,
            "destination": 1,
            "weight": int(output_info.iloc[0, 1]),
            "quality": [
                {"parameter": "Fe", "minimum": 60, "maximum": 100, "goal": 65, "importance": 10},
                {"parameter": "SiO2", "minimum": 2.8, "maximum": 5.8, "goal": 5.8, "importance": 1000},
                {"parameter": "Al2O3", "minimum": 2.5, "maximum": 4.9, "goal": 4.9, "importance": 100},
                {"parameter": "P", "minimum": 0.05, "maximum": 0.07, "goal": 0.07, "importance": 100},
            ],
            "time": 600
        }
    ]

    inputs = [
        {
            "id": 1,
            "weight": 0.0,
            "quality": [
                {"parameter": "Fe", "value": 60},
                {"parameter": "SiO2", "value": 1.5},
                {"parameter": "Al2O3", "value": 0.8},
                {"parameter": "P", "value": 1.0}
            ],
            "time": 8.0
        }
    ]

    # Build the instance data structure
    instance_data = InstanceDataBuilder.build_instance_data(stockpiles, engines_df, travel_times, inputs, outputs)

    # Write the data to a JSON file
    with open(instance_json_path, "w") as json_file:
        json.dump(instance_data, json_file, indent=2)

    run_pyblend_command(Path(instance_json_path).name, output_json_path)

    operations_df, outputs_df_out = json_input_output_to_excel(
        instance_data,
        output_json_path,
    )
    update_excel_sheets(operations_df, outputs_df_out)

In [14]:
main(
    "/Users/erikingwersen/Desktop/fileshare/out/excel/out_interactive.xlsm",
    "../tests/instance_interactive.json",
    "../out/json/out_interactive.json",
)

In [29]:
operations_df, outputs_df_out = json_input_output_to_excel(
    "../tests/instance_interactive.json",
    "../out/json/out_interactive.json",
)

/var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/ipykernel_17442/1283604484.py:296: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  engines_df[from_to_df["from_to"].to_list()] = -1
/var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/ipykernel_17442/1283604484.py:296: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  engines_df[from_to_df["from_to"].to_list()] = -1
/var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/ipykernel_17442/1283604484.py:296: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `

In [30]:
operations_df

,Veículo,Pilha,Peso Inicial Pilha,Peso Final Pilha,Carregamento (ton),Início,Fim,Tempo Carregamento,Tempo Deslocamento,Fe,SiO2,Al2O3,P
0,1,11,45000,0.0,45000,0.90,12.44,11.54,0.90,65.20,4.1,4.9,0.05
1,1,8,45000,0.0,45000,13.94,25.48,11.54,1.50,60.30,5.5,4.7,0.06
2,1,17,62500,0.0,62500,25.64,41.67,16.03,0.16,68.10,3.6,4.8,0.06
3,1,20,70000,0.0,70000,42.88,60.83,17.95,1.21,65.90,4.4,5.2,0.09
4,1,6,75000,0.0,75000,61.93,81.16,19.23,1.10,65.40,4.5,4.9,0.06
5,2,19,67500,66250.0,1250,0.70,1.18,0.48,0.70,69.40,4.0,5.1,0.08
6,2,9,50000,0.0,50000,1.34,20.57,19.23,0.16,68.30,3.2,4.8,0.07
7,2,1,60000,58750.0,1250,21.27,21.75,0.48,0.70,10.30,2.3,4.5,0.05
8,2,2,45000,0.0,45000,22.65,39.96,17.31,0.90,76.30,5.6,4.8,0.05
9,2,18,65000,0.0,65000,40.76,65.76,25.00,0.80,67.00,3.9,4.9,0.07


In [27]:
excel_file = "/Users/erikingwersen/Desktop/fileshare/out/excel/out_interactive.xlsm"
wb = xw.Book(excel_file)

resultados_sheet = wb.sheets['Resultados']

In [38]:
operations_df = operations_df.fillna('')
operations_df.iloc[-1, 0] = operations_df.iloc[0, -1].astype(str)

In [39]:
resultados_sheet['A1'].value = operations_df

In [43]:
update_excel_sheets(operations_df, outputs_df_out, "/Users/erikingwersen/Desktop/fileshare/out/excel/out_interactive.xlsm")

In [21]:
outputs_df_out

,Elemento,Valor,Mínimo,Máximo,Meta,Check
0,Fe,67.45,60.00,100.00,65.00,True
1,SiO2,4.30,2.80,5.80,5.80,True
2,Al2O3,4.90,2.50,4.90,4.90,True
3,P,0.07,0.05,0.07,0.07,True
